### Import Necessary Libraries

In [10]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session

### Define S3 Bucket Name and Region

In [15]:
bucket_name="bank-app-560"
my_region=boto3.session.Session().region_name
print(my_region) # Print the current AWS region

us-east-2


### Create S3 Bucket

In [16]:

import boto3

s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        # Create bucket without LocationConstraint for us-east-1
        s3.create_bucket(Bucket=bucket_name)
    else:
         # Create bucket with LocationConstraint for other regions
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': my_region
            }
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)


S3 bucket created successfully


### Define Output Path

In [17]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bank-app-560/xgboost-as-a-built-in-algo/output


### Download and Load Dataset

In [18]:

import pandas as pd
import urllib

# Download the dataset
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

# Load dataset into a DataFrame
try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


### Train-Test Split

In [19]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


### Save Training Data to S3

In [21]:
# Save training data to CSV and upload to S3
pd.concat([train_data['y_yes'],train_data.drop(['y_no','y_yes'],
                                              axis=1)],
                                                 axis=1).to_csv('train.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')


# Define the S3 input for training
s3_input_train=sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name,prefix),content_type='csv')

[12/29/24 18:06:32] INFO     Found credentials from IAM Role:                                   ]8;id=30908;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=802768;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### Import Training Input

In [23]:
from sagemaker.inputs import TrainingInput


### Save Test Data to S3

In [25]:
# Save test data to CSV and upload to S3
pd.concat([test_data['y_yes'], test_data.drop(['y_no','y_yes'],axis=1)],axis=1).to_csv('test.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')

# Define the S3 input for testing
s3_input_test=sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name,prefix),content_type='csv')

[12/29/24 20:03:20] INFO     Found credentials from IAM Role:                                   ]8;id=837971;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=147167;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### Retrieve XGBoost Container

In [26]:
# Retrieve the XGBoost container image URI
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

[12/29/24 22:22:57] WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=277704;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=239448;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Defaulting to only available Python version: py3                     ]8;id=144664;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=500177;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#608\608]8;;\

[12/29/24 22:22:58] INFO     Defaulting to only supported image scope: cpu.                       ]8;id=399426;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=915005;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#532\532]8;;\

### Define Hyperparameters

In [27]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

### Create SageMaker Estimator

In [30]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

### Train the Model

In [31]:
# Start model training
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

[12/29/24 23:02:50] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=191300;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=276380;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=575758;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=629689;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2024-12-29-23-02-50-487                                             

2024-12-29 23:02:51 Starting - Starting the training job...
2024-12-29 23:03:05 Starting - Preparing the instances for training...
2024-12-29 23:03:42 Downloading - Downloading the training image...
2024-12-29 23:04:18 Training - Training image download completed. Training in progress...[2024-12-29 23:04:34.041 ip-10-0-229-179.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single n

### Deploy the Model

In [40]:
# Deploy the trained model
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

[12/30/24 00:00:48] INFO     Creating model with name: sagemaker-xgboost-2024-12-30-00-00-48-849    ]8;id=256626;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=163588;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[12/30/24 00:00:54] INFO     Creating endpoint-config with name                                     ]8;id=580086;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=729726;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             sagemaker-xgboost-2024-12-30-00-00-48-849                                             

                    INFO     Creating endpoint with name sagemaker-xgboost-2024-12-30-00-00-48-849  ]8;id=779550;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=828192;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!

### Make Predictions

In [41]:

from sagemaker.serializers import CSVSerializer

# Prepare test data and make predictions
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer =CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [36]:

predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

### Evaluate Model Performance

In [37]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



### Cleanup Resources

In [42]:

sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[12/30/24 00:21:08] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=939420;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=960976;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Deleting endpoint with name: sagemaker-xgboost-2024-12-30-00-00-48-849 ]8;id=830433;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=811886;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\

[{'ResponseMetadata': {'RequestId': 'PQ1BQGRGA2EFP48X',
   'HostId': 'nLc/wE3vBj/NXjMWzYTcoM7/aIwvGB74/sYsA8Uf9fWVh7E0QrDcPEsZRaUGuHivlNNWsCZl9vMFUWnyb/6Iahk/hpXF7ooVycp9PS5wtxQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'nLc/wE3vBj/NXjMWzYTcoM7/aIwvGB74/sYsA8Uf9fWVh7E0QrDcPEsZRaUGuHivlNNWsCZl9vMFUWnyb/6Iahk/hpXF7ooVycp9PS5wtxQ=',
    'x-amz-request-id': 'PQ1BQGRGA2EFP48X',
    'date': 'Mon, 30 Dec 2024 00:21:09 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-12-29-23-02-50-487/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-12-29-23-02-50-487/profiler-output/framework/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-12-29-23-02-50-487/debug-output/events/000